In [1]:
import numpy as np
import pandas as pd
%matplotlib inline

In [2]:
nn_pred = pd.read_csv('submission_keras_shift_perm.csv')
nn_pred.rename(columns={'loss':'loss_nn'}, inplace=True)

In [3]:
nn_meta = pd.read_csv('preds_oob.csv')
nn_meta.rename(columns={'loss':'loss_nn'}, inplace=True)

In [4]:
xgb_pred = pd.read_csv('submission_5fold-average-xgb_fairobj_1131.430646_2016-11-24-05-21.csv')
xgb_pred.rename(columns={'loss':'loss_xgb'}, inplace=True)

In [5]:
xgb_meta = pd.read_csv('metafeature_10fold_raw.csv')
xgb_meta.rename(columns={'loss':'loss_xgb'}, inplace=True)

In [6]:
xgb_pred2 = pd.read_csv('sub_v.csv')
xgb_pred2.rename(columns={'loss':'loss_xgb2'}, inplace=True)

In [7]:
xgb_meta2 = pd.read_csv('sub_v_train.csv')
xgb_pred2.rename(columns={'loss':'loss_xgb2'}, inplace=True)

In [8]:
gbm_pred = pd.read_csv('gbm_first.csv')
gbm_pred.rename(columns={'loss':'loss_gbm'}, inplace=True)

In [9]:
gbm_meta = pd.read_csv('metafeature_10fold_gbm.csv')
gbm_pred.rename(columns={'loss':'loss_gbm'}, inplace=True)

In [60]:
total_pred = nn_pred.merge(xgb_pred, on='id').merge(xgb_pred2, on='id').merge(gbm_pred, on='id')

In [61]:
total_meta = nn_meta.merge(xgb_meta, on='id').merge(xgb_meta2, on='id').merge(gbm_meta, on='id')

In [62]:
total_pred.head()

,id,loss_nn,loss_xgb,loss_xgb2,loss_gbm
0,4,1702.271565,1609.509766,1516.827881,1567.447644
1,6,1795.501449,1935.612915,1994.615967,2006.551270
2,9,9010.045728,9515.419922,8897.466797,9222.415633
3,12,7460.658379,6272.907227,5890.556152,6481.133755
4,15,757.285362,780.405090,796.562317,799.944744


In [63]:
total_meta.head()

,id,loss_nn,meta_xgb,loss_x,loss_y
0,106313,1070.906348,1095.242065,1182.481201,1141.351672
1,136287,1658.718567,1733.081543,1907.467285,1941.156353
2,11041,1826.117810,1922.265137,1697.623291,1843.919503
3,313015,3371.868506,3158.239258,3350.754150,3198.897750
4,188200,10972.408203,8422.389648,9041.215820,8734.985661


In [64]:
total_meta.sort_values(by='id', inplace=True)
total_meta.head()

,id,loss_nn,meta_xgb,loss_x,loss_y
180536,1,1843.111414,2041.131104,2016.575439,2006.982265
114129,2,1644.979285,1758.783813,1744.768799,1772.275405
64050,5,3728.797046,4228.701172,4161.208984,4269.090052
166274,10,1001.145459,1095.230957,1027.500488,1064.597396
179990,11,3243.387695,3223.909668,3009.711914,3361.836242


In [65]:
total_meta = total_meta.drop(['loss_y'], axis=1)
total_pred = total_pred.drop(['loss_gbm'], axis=1)

In [ ]:
#total_meta = total_meta.iloc[:,:3]
#total_pred = total_pred.iloc[:,:3]

In [ ]:
#average = pd.DataFrame({'id':total_pred['id'], 'loss':total_pred.iloc[:,1:].mean(axis=1)})
#average.to_csv('avr_4_models.csv', index=False)

In [ ]:
#pd.DataFrame({'id':d['id'], 'loss':d.iloc[:,1:].median(axis=1)}).to_csv('median.csv', index=False)

In [ ]:
#pd.DataFrame({'id':d['id'], 'loss':0.4*d.iloc[:,1]+0.35*d.iloc[:,2]+0.25*d.iloc[:,3]}).to_csv('weight.csv', index=False)

In [ ]:
#pd.DataFrame({'id':d['id'], 'loss':0.35*d.iloc[:,1]+0.4*d.iloc[:,2]+0.25*d.iloc[:,3]}).to_csv('weight3.csv', index=False)

In [16]:
import os
from pylightgbm.models import GBMRegressor
os.environ['LIGHTGBM_EXEC'] = r'D:\LightGBM\LightGBM\windows\x64\Release\lightgbm.exe'

In [17]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold, train_test_split

In [18]:
shift = 190

In [66]:
train_test = pd.read_csv(r'D:/allstate/train_test.csv').drop('Unnamed: 0', axis=1)
directory = r"D:/allstate/"
ids = pd.read_csv(directory + 'test.csv.zip')['id']
ntrain = 188318
train_test[train_test['id']==4]

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cat14_cat38,cat14_cat24,cat14_cat82,cat14_cat25,cat38_cat24,cat38_cat82,cat38_cat25,cat24_cat82,cat24_cat25,cat82_cat25
188318,4,1,2,1,1,1,1,1,1,2,...,27,27,28,27,27,28,27,28,27,53


In [67]:
train = train_test[:ntrain]
test = train_test[ntrain:]

In [68]:
from sklearn.cluster import MiniBatchKMeans

In [69]:
clust = MiniBatchKMeans(max_iter=300, n_clusters=5, batch_size=5000, max_no_improvement=20)
clust.fit(train.drop(['id','loss'], axis=1))

MiniBatchKMeans(batch_size=5000, compute_labels=True, init='k-means++',
        init_size=None, max_iter=300, max_no_improvement=20, n_clusters=5,
        n_init=3, random_state=None, reassignment_ratio=0.01, tol=0.0,
        verbose=0)

In [70]:
train['clust'] = clust.predict(train.drop(['id','loss'], axis=1))
test['clust'] = clust.predict(test.drop(['id','loss'], axis=1))

C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [71]:
train = train.merge(total_meta, on='id')

In [72]:
#train.iloc[:,-4:] = np.log(train.iloc[:,-4:] + shift)

In [73]:
test = test.merge(total_pred, on='id')

In [74]:
#test.iloc[:,-4:] = np.log(test.iloc[:,-4:] + shift)

In [75]:
train.shape, test.shape

((188318, 731), (125546, 731))

In [76]:
train_loss = train.loss
train_x = train.drop(['id','loss'], axis=1)
test_x = test.drop(['id','loss'], axis=1)

In [78]:
train_x.iloc[:,-3:] = np.log(train_x.iloc[:,-3:] + shift)
test_x.iloc[:,-3:] = np.log(test_x.iloc[:,-3:] + shift)

In [79]:
train_y = np.log(train_loss + shift)

In [80]:
X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, test_size=0.1, random_state=0)

In [38]:
#from sklearn.decomposition import TruncatedSVD

In [39]:
#svd = TruncatedSVD(n_iter=10, n_components=75)
#svd.fit(X_train)

In [40]:
#svd.explained_variance_ratio_.sum()

In [41]:
#X_train_svd = svd.transform(X_train)
#X_val_svd = svd.transform(X_val)

In [81]:
clf = GBMRegressor(application='regression',
                   num_iterations=5000,
                   learning_rate=0.01,
                   num_leaves=5,
                   max_bin=900,
                   tree_learner='serial',
                   num_threads=8,
                   min_data_in_leaf=10,
                   metric='l1',
                   feature_fraction=0.6,
                   bagging_fraction=1.0,
                   early_stopping_round=50,
                   max_depth=-1,
                   boosting_type='gbdt',
                   verbose=False)
clf.fit(X_train, y_train, test_data=[(X_val, y_val)])

In [82]:
clf.best_round

5000

In [83]:
predicted = clf.predict(X_val)
mean_absolute_error(np.exp(y_val), np.exp(predicted))

787.77050913413314

In [ ]:
1131.2147677230632

In [ ]:
1119

1132

1107

1113.3251491

1124.630568

1127.3536544333508

In [ ]:
#test_x_svd = svd.transform(test_x)
out = clf.predict(test_x)
out = np.exp(out)-shift

In [ ]:
stacking_pred = pd.DataFrame({'id':ids, 'loss':out})
stacking_pred.to_csv('stacking_pred.csv', index=False)

In [ ]:
n_folds = 10
cv_sum = 0
early_stopping = 50
mpred = np.zeros(test_x.shape[0])

In [ ]:
metafeature = pd.DataFrame()
kf = KFold(n_folds)
for i, (train_index, valid_index) in enumerate(kf.split(train_x)):
    print('\n Fold %d' % (i+1))
    X_train, X_val = train_x.iloc[train_index], train_x.iloc[valid_index]
    y_train, y_val = train_y.iloc[train_index], train_y.iloc[valid_index]
    
    clf = GBMRegressor(application='regression',
                       num_iterations=5000,
                       learning_rate=0.005,
                       num_leaves=6,
                       max_bin=900,
                       tree_learner='serial',
                       num_threads=8,
                       min_data_in_leaf=10,
                       metric='l1',
                       feature_fraction=0.6,
                       bagging_fraction=1.0,
                       early_stopping_round=50,
                       max_depth=-1,
                       boosting_type='gbdt',
                       verbose=False)
    clf.fit(X_train, y_train, test_data=[(X_val, y_val)])
    
    scores_val = clf.predict(X_val)
    cv_score = mean_absolute_error(np.exp(y_val), np.exp(scores_val))
    print('eval-MAE: %.6f' % cv_score)
    cv_sum += cv_score
    y_pred = np.exp(clf.predict(test_x)) - shift
    #stacking    
    metafeature2 = pd.DataFrame()
    y_stack = np.exp(scores_val) - shift
    metafeature2['id'] = train.iloc[valid_index]['id']
    metafeature2['loss'] = y_stack
    metafeature = pd.concat([metafeature, metafeature2])
    mpred += y_pred   
mpred /= n_folds

In [ ]:
print('eval-MAE: %.6f' % (cv_sum/n_folds))

In [ ]:
print("Writing results")
result = pd.DataFrame(mpred, columns=['loss'])
result["id"] = ids
result = result.set_index("id")
result.to_csv('gbm_stack.csv', index=True, index_label='id')

In [ ]:
import gc

In [ ]:
gc.collect()

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
forest = RandomForestRegressor(n_jobs=8, n_estimators=75)
forest.fit(X_train, y_train)

In [ ]:
fpred = forest.predict(X_val)
mean_absolute_error(np.exp(y_val), np.exp(fpred))

In [ ]:
fpred, predicted

In [ ]:
y_val.values